# Web Sraping

**Goal**: Retrieve articles from *ekathimerini.com*

To extend the use of this .ipynb for other sites we only need to change the cssSelectors depending on the layout of the Page

END RESULTS IN [GDRIVE Folder](https://drive.google.com/drive/folders/1rde9viDnmkJIRKaLTsGHufBVIMw2q3fl?usp=sharing)

## Load Drive
Load google drive for storage

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup
1. install selenium
2. update and install chromedriver 
3. insert chromedriver to $PATH
4. Basic python imports 
5. chromedriver options setup

In [2]:
!pip install selenium

     |████████████████████████████████| 968 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 37.3 MB/s 
     |████████████████████████████████| 359 kB 28.1 MB/s 
     |████████████████████████████████| 4.0 MB 41.3 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,575 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Hit:10 http://ppa.launchpad.net/cran/lib

In [4]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [5]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

## Generate sitemap.xml post urls
Used to parse this [page](https://www.ekathimerini.com/sitemap.xml) </br>
We generate a list of all sitemap.xml urls based on dates we are interested in. Dates of interest range from 01-01-2019 to 12-04-2022 

In [7]:
'''
 Find all links of sitemap 
 all Articles posted in 01-2019 can be found on this sample link
 https://www.ekathimerini.com/sitemap-pt-post-2019-01.xml
'''


baseUrl = 'https://www.ekathimerini.com/sitemap-pt-post-'

dates=[]
years=[2019,2020,2021,2022]
for i in range(1,13):
  for year in years:
    if year==2022 and i>=5:
      continue
    else:
      dates.append(str(year)+'-'+str(i).zfill(2))
  
urls = [baseUrl+date+'.xml' for date in dates]
print(urls)


['https://www.ekathimerini.com/sitemap-pt-post-2019-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2019-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2019-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2019-04.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-04.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-04.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-04.xml', 'https://www.ekathimerini.com/sitemap-p

In [ ]:
print(len(urls))

40


## Iterate through all post-xml pages
We use the urls generated above. For every url we extract all the article urls contained in that particular post-xml url. All articles are then stored in  [google drive folder](https://drive.google.com/drive/folders/1rde9viDnmkJIRKaLTsGHufBVIMw2q3fl?usp=sharing)

In [ ]:

allUrls=[]

for index,url in enumerate(urls):
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get(url)
  total=driver.find_elements(By.TAG_NAME,"a")
  urlOI=[]
  for idx, val in enumerate(total):
    if 'kathimerini' in val.text:
      urlOI.append(val.text)
    else:
      continue
  
  
  #Run once
  #Remove X to rerun
  with open(".X/drive/MyDrive/webScraping/kathimerini/links/file"+dates[index]+".txt", "w") as output:
    for item in urlOI:
        output.write("%s\n" % item)
  

## Parse Links of Interest and Extract corpus

Now we iterate through every saved .txt file (which contains links based on date of publishment). We only get all the links per month from 01-2019 up until 04-2022. In total we have 41009  articles ( some of them do not have a corpus or a title so they are not stored in the .csv file, approx. runtime to create that .csv is ~35-40 minutes per month in google colab)

In [8]:
# Used for .csv writing (csv writer treated strings as iterables and I had to find a workaround)
class non_iter_str(str):
    def __iter__(self):
        yield self

In [ ]:
# Find number of articles listed in ekathimerini
numArticles=0
for i in range(len(dates)):
  with open("./drive/MyDrive/webScraping/kathimerini/links/file"+dates[i]+".txt", "r") as input:
    inpt = input.readlines()  
    numArticles+=len(inpt)
    #print(len(inpt))

print(numArticles)


41019


In [ ]:
'''
Create a header for .csv
Run only the first time
'''
import csv
import numpy as np

#Create the header for .csv file
#Remove X in order to rerun
with open("X./drive/MyDrive/webScraping/kathimerini/corpus_new.csv", "w") as file:
        header=[non_iter_str('url'),non_iter_str('date'),non_iter_str('title'),non_iter_str('corpus')]
        write = csv.writer(file)
        write.writerows([header])


In [30]:
'''
In case colab's kernel shuts down, run this to find already Parsed articles
'''

# Find already parsed articles to avoid re-parsing
import pandas as pd 
import csv 
import numpy as np
df = pd.read_csv("./drive/MyDrive/webScraping/kathimerini/corpus_new_header.csv") 

alreadyParsed = set(list(df.iloc[:, 0]))

In [32]:
print(len(alreadyParsed))

41009


In [11]:
nanVals = 0
nanValsTitle=0
for w in range(1,len(dates)):
  
  with open("./drive/MyDrive/webScraping/kathimerini/links/file"+dates[w]+".txt", "r") as input:
    print(nanVals,nanValsTitle)
  
    print("Currently at ",dates[w])
    
    #Check for NaN values at corpus and title
    nanVals = 0
    nanValsTitle=0
    print(nanVals,nanValsTitle)

    # Prevent some issues regarding chromedriver sync
    sleep(0.05) 
    
    # Read all articles in .txt file
    # inpt variable is an iterable which contains every url of interest
    inpt = input.readlines()  
    

    # Get 100 evenly spaced articles  
    #indices = np.round(np.linspace(0,len(inpt)-1,100)).astype(int)
    #print(indices)
    
    #totalData list will be used to accumulate all needed information before
    #writing them to the .csv file
    totalData = []
    
    for i in range(len(inpt)):
      
      #URL of interest
      tempUrl = inpt[i]

      #Check if url of interest exists in our dataset
      if tempUrl in alreadyParsed:
        continue
    
      
      # Extract article id from url (later used in css selector)
      idArticle=str([int(s) for s in tempUrl.split("/") if s.isdigit()][0])
      

      #Initiliaze driver
      driver = webdriver.Chrome('chromedriver',options=chrome_options)
      driver.get(tempUrl)
     
      
      
      #Get Corpus (cssSelector is found manually if we inspect a page but remains the same for every article)
      cssSelector = '#post-'+str(idArticle)+' > div.entry-content'
      #We use try because there are a few (insignificant) articles that have a peculiar css layout 
      try:
        corpusElement= driver.find_element(by=By.CSS_SELECTOR, value=cssSelector)
      except:
        nanVals+=1
        continue
      
      
      #Get Date
      dateCssSelector = '#post-'+str(idArticle)+'> div.entry-meta > div > span > time.meta-date-modified.updated'
      dateElement = driver.find_element(by=By.CSS_SELECTOR, value=dateCssSelector) 
      
      #Get title
      titleCssSelector = '#post-'+str(idArticle)+' > header > h1 '
      try:
        titleElement = driver.find_element(by=By.CSS_SELECTOR, value=titleCssSelector) 
      except:
        nanValsTitle+=1
        continue
      
    
      # csv has the following format url,date,title,corpus
      #non_iter_str() is used because there was a problem because strings in python are iterable
      data=(non_iter_str(tempUrl),non_iter_str(dateElement.text[0:10]),non_iter_str(titleElement.text),non_iter_str(corpusElement.text))
      totalData.append(data)

      #Used for progress information
      if i%100==0:
        print(dates[w],": ",len(inpt)-i, " remaining")
    

    # Write all articles of a specific date (Month-Year) to the .csv file 
    with open("./drive/MyDrive/webScraping/kathimerini/corpus_new.csv", "a+") as file:
      for j in range(len(totalData)):
        write = csv.writer(file)
        write.writerows([totalData[j]])


0 0
Currently at  2020-01
0 0
0 0
Currently at  2021-01
0 0
0 0
Currently at  2022-01
0 0
0 0
Currently at  2019-02
0 0
0 0
Currently at  2020-02
0 0
0 0
Currently at  2021-02
0 0
0 2
Currently at  2022-02
0 0
0 0
Currently at  2019-03
0 0
0 0
Currently at  2020-03
0 0
1 0
Currently at  2021-03
0 0
0 0
Currently at  2022-03
0 0
0 0
Currently at  2019-04
0 0
1 0
Currently at  2020-04
0 0
1 0
Currently at  2021-04
0 0
0 0
Currently at  2022-04
0 0
0 0
Currently at  2019-05
0 0
0 0
Currently at  2020-05
0 0
1 0
Currently at  2021-05
0 0
0 0
Currently at  2019-06
0 0
0 0
Currently at  2020-06
0 0
1 0
Currently at  2021-06
0 0
0 0
Currently at  2019-07
0 0
0 0
Currently at  2020-07
0 0
1 0
Currently at  2021-07
0 0
0 0
Currently at  2019-08
0 0
0 0
Currently at  2020-08
0 0
0 0
Currently at  2021-08
0 0
0 0
Currently at  2019-09
0 0
0 0
Currently at  2020-09
0 0
0 0
Currently at  2021-09
0 0
0 0
Currently at  2019-10
0 0
0 0
Currently at  2020-10
0 0
0 0
Currently at  2021-10
0 0
0 0
Curren

In [14]:
#Used to test if .csv has the proper output
import pandas as pd
df = pd.read_csv('./drive/MyDrive/webScraping/kathimerini/corpus_new.csv')  
df

,https://www.ekathimerini.com/sports/237244/paok-and-aek-scrape-through-as-reds-march-on/\n,31.01.2019,PAOK and AEK scrape through as Reds march on,"PAOK and AEK scored narrow home wins for the Super League on Wednesday ahead of their clash on Sunday in Athens, while Olympiakos thrashed visiting Larissa and Panathinaikos won at Panetolikos.\nKarol Swiderski calmed PAOK’s nerves scoring three minutes from time for a 2-1 victory over PAS Giannina at Toumba. Dimitris Pelkas had given PAOK a first-half lead from the penalty spot before PAS equalized with a Dimitris Manos goal that should have been ruled offside. PAOK also had a penalty kick saved.\nOlympiakos stayed eight points behind PAOK with an emphatic 4-0 over Larissa in Piraeus, thanks to a Costas Fortounis hat trick and a Giorgos Masouras opener, his first goal in a red jersey.\nAEK had its top scorer, the Argentine Ezequiel Ponce, on target in its 1-0 home win over Levadiakos. It is 14 points off the pace, and six above fourth-placed Atromitos that drew 1-1 at OFI Crete on Thursday.\nPanathinaikos is now alone in fifth, three points behind Atromitos, courtesy of a 1-0 win at Panetolikos. Yiannis Bouzoukis gave the Greens their first league win in 2019 on Wednesday, despite playing with 10 men for over half an hour.\nAris scored an important 1-0 win at Xanthi on Thursday to leapfrog its host and reach the sixth spot.\nThere were also home wins for Lamia (2-1 over Apollon Smyrnis) and Panionios (1-0 over Asteras Tripolis).\nSOCCER"
0,https://www.ekathimerini.com/economy/237242/lo...,31.01.2019,Local startups attracting investments,The Greek startup ecosystem has been showing s...
1,https://www.ekathimerini.com/economy/237241/gr...,31.01.2019,Greek tankers have loads confiscated in Venezuela,Greek shipowners face the prospect of not bein...
2,https://www.ekathimerini.com/economy/237239/wa...,31.01.2019,Warm wind beneath new bond’s wings,The markets appear particularly keen on the ne...
3,https://www.ekathimerini.com/economy/237238/at...,31.01.2019,ATHEX: Monthly rise of 3.5 pct for benchmark,January ended with monthly gains at the Greek ...
4,https://www.ekathimerini.com/news/237237/ankar...,31.01.2019,Ankara raises issue of eight officers again,A few days before Prime Minister Alexis Tsipra...
...,...,...,...,...
41003,https://www.ekathimerini.com/culture/1172758/p...,01.12.2021,Plans to guard monuments against climate change,A working group of the Culture Ministry has st...
41004,https://www.ekathimerini.com/economy/1172814/t...,01.12.2021,Terna to issue €300 mln common bond loan,GEK Terna on Tuesday announced the issue of a ...
41005,https://www.ekathimerini.com/economy/1172856/p...,01.12.2021,Post-Weidmann ECB might be more lenient,Future support from the European Central Bank ...
41006,https://www.ekathimerini.com/nytimes/1172790/f...,01.12.2021,Fragmented reactions hinder global fight again...,ROME — In a wrenchingly familiar cycle of trac...
